In [16]:
'''Import packages for loading data, analysing, and plotting'''

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xesmf as xe
%matplotlib inline
import cartopy
import cartopy.crs as ccrs
import matplotlib
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy.ma as ma
import math
import os
import matplotlib.colors as colors
import scipy
from matplotlib.projections import PolarAxes
import mpl_toolkits.axisartist.floating_axes as FA
import mpl_toolkits.axisartist.grid_finder as GF

In [17]:
#Define PMIP generations and model names
pmip_v4='PMIP4'
pmip_v3='PMIP3'
pmip={}
pmip['PMIP4']=['ACCESS-ESM1-5',
               'AWI-ESM-1-1-LR',
               'CESM2',
               'CNRM-CM6-1',
               'EC-Earth3-LR',
               'FGOALS-f3-L',
               'FGOALS-g3',
               'GISS-E2-1-G',
               'HadGEM3-GC31-LL',
               'HadGEM3-GC31',
               'INM-CM4-8',
               'IPSL-CM6A-LR',
               'MIROC-ES2L',
               'MPI-ESM1-2-LR',
               'MRI-ESM2-0',
               'NESM3',
               'NorESM1-F',
               'NorESM2-LM',
               'UofT-CCSM-4']
pmip['PMIP3']=['BCC-CSM1-1',
               'CCSM4',
               'CNRM-CM5',
               'COSMOS-ASO',
               'CSIRO-Mk3L-1-2',
               'CSIRO-Mk3-6-0',
               'EC-EARTH-2-2',
               'FGOALS-g2',
               'FGOALS-s2',
               'GISS-E2-1-G',
               'GISS-E2-R',
               'HadCM3',
               'HadGEM2-CC',
               'HadGEM2-ES',
               'IPSL-CM5A-LR',
               'KCM1-2-2'
               'MIROC-ESM',
               'MPI-ESM-P',
               'MRI-CGCM3']

In [25]:
#No change needs here

'''Define calculating functions'''

#This function will get all available experiment names
def experimentlist():
    exps=[]
    file_path = "../data" 
    for dirpaths, dirnames, filenames in os.walk(file_path):
        for d in dirnames:
            exps.append(d)
    return exps

#This function will get all available model names in the experiment 
def modellist(experiment_name):
    models=[]
    file_path = "../data/%s" %(experiment_name)
    for dirpaths, dirnames, filenames in os.walk(file_path):
        for f in filenames:
            mname=f.split("_")[0]
            models.append(mname)
    return models

#This function will get all available filenames in the experiment 
def filenamelist(experiment_name):
    filenames=[]
    file_path = "../data/%s" %(experiment_name)
    for dirpaths, dirnames, files in os.walk(file_path):
        for f in files:
            ff='../data/%s/%s'%(experiment_name,f)
            filenames.append(ff)
    return filenames

#This function will identify models in the ensemble
def identify_ensemble_members(variable_name,experiment_name):
    datadir="../data/%s" %(experiment_name)
    ensemble_members=!../scripts/find_experiment_ensemble_members.bash {experiment_name} {variable_name} {datadir}
    return ensemble_members

#This function will list excat model name
def extract_model_name(filename):
    file_no_path=filename.rpartition("/")
    file_strings=file_no_path[2].partition("_")
    model_name=file_strings[0]
    return model_name

def ensemble_members_dict(variable_name,experiment_name):
    ens_mems=identify_ensemble_members(variable_name,experiment_name)
    ens_mems_dict={extract_model_name(ens_mems[0]):ens_mems[0]}
    for mem in ens_mems[1:]:
        ens_mems_dict[extract_model_name(mem)]=mem
    return ens_mems_dict

experimentlist()

['midHolocene-cal-adj',
 'atl3',
 'lig127k',
 'midHolocene',
 'barchart',
 'lig127-cal-adj',
 'HadISST',
 'piControl',
 'gpcp',
 'amm',
 'historical',
 '1pctCO2',
 'abrupt4xCO2',
 'midPliocene-eoi400',
 'reanalysis',
 'observations',
 'lgm',
 'lgm-cal-adj']

In [19]:
modellist('HadISST')

['UofT-CCSM-4',
 'MIROC-ES2L',
 'NESM3',
 'CNRM-CM6-1',
 '.DS',
 'MPI-ESM1-2-LR',
 'IPSL-CM6A-LR',
 'HadGEM3-GC31-LL',
 'NorESM1-F',
 'AWI-ESM-1-1-LR',
 'HadGEM3-GC31',
 'CESM2',
 'FGOALS-f3-L',
 'MRI-ESM2-0',
 'EC-Earth3-LR',
 'FGOALS-g3',
 'INM-CM4-8',
 'GISS-E2-1-G',
 'NorESM2-LM',
 'ACCESS-ESM1-5']

In [20]:
modellist('piControl')

['GISS-E2-1-G',
 'FGOALS-g3',
 'COSMOS-ASO',
 'NESM3',
 'HadGEM3-GC31-LL',
 'EC-Earth3-LR',
 'INM-CM4-8',
 'MRI-CGCM3',
 'CNRM-CM5',
 'FGOALS-g2',
 'ACCESS-ESM1-5',
 'GISS-E2-R',
 'EC-EARTH-2-2',
 'NorESM2-LM',
 'HadGEM2-ES',
 'IPSL-CM5A-LR',
 'MRI-ESM2-0',
 'MIROC-ES2L',
 'UofT-CCSM-4',
 'IPSL-CM6A-LR',
 'CESM2',
 'AWI-ESM-1-1-LR',
 'FGOALS-f3-L',
 'MPI-ESM1-2-LR',
 'KCM1-2-2',
 'BCC-CSM1-1',
 'HadGEM2-CC',
 'HadCM3',
 'MPI-ESM-P',
 'CCSM4',
 'CSIRO-Mk3L-1-2',
 'NorESM1-F',
 'CSIRO-Mk3-6-0',
 'FGOALS-s2',
 'CNRM-CM6-1',
 'MIROC-ESM']

In [36]:
#ensemble difference for HadISST
def ensemble_diffence(pmip_v,experiment_name,variable_name,filename):
    modellist=[]
    lats=np.arange(-89.5, 90., 1.0)
    lons=np.arange(-179.5, 180, 1.0)
    grid_1x1= xr.Dataset({'lat': (['lat'],lats),
                         'lon': (['lon'],lons)})
    # Use the bash script to find files with the specified variable
    A_dict=ensemble_members_dict(variable_name,experiment_name)
    B_dict=ensemble_members_dict(variable_name,'HadISST')

    #how many of the models are from this generation?
    n_gcms_of_pmip_v=0
    for gcm in A_dict:
        if gcm in B_dict:
            if gcm in pmip[pmip_v]:
                n_gcms_of_pmip_v=n_gcms_of_pmip_v+1

    #Set up the holding array
    gcms=np.arange(0,n_gcms_of_pmip_v,1)  
    data=np.full((n_gcms_of_pmip_v,len(lats),len(lons)),np.nan)
 
    #loop over models
    n=0
    for gcm in A_dict:
        if gcm in B_dict:
            if gcm in pmip[pmip_v]:
                modellist.append(gcm)
                expt_a_file=xr.open_dataset(A_dict.get(gcm),decode_times=False)
                expt_a=expt_a_file[variable_name]
                expt_b_file=xr.open_dataset(B_dict.get(gcm),decode_times=False)
                expt_b=expt_b_file[variable_name]
                diff=expt_a-expt_b
                this_regridder=xe.Regridder(expt_a_file,grid_1x1,'bilinear', reuse_weights=False,periodic=True) 
                diff_1x1=this_regridder(diff) 
                expt_a_file.close()
                expt_b_file.close()
                data[n,:,:]=diff_1x1
                n=n+1

    #write out dataset        
    ds=xr.Dataset({variable_name:(["gcms","lat","lon"],data),
                   "modellist":(["gcms"],modellist)},
                 coords={'lat': (['lat'],lats),
                         'lon': (['lon'],lons),
                         'gcms':(['gcms'],gcms)})
    ds.to_netcdf(filename)      

    return modellist

In [37]:
#piControl - sst

#diff_sst_spatialmean_djf

filename='../outputs/netcdf/diff_sst/PMIP4_piControl_diff_sst_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='piControl'
variable_name='sst_spatialmean_djf'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)
diff_list

#diff_sst_spatialmean_jja
filename='../outputs/netcdf/diff_sst/PMIP4_piControl_diff_sst_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='piControl'
variable_name='sst_spatialmean_jja'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)
diff_list


AssertionError: The horizontal shape of input data is (36, 24), different from that ofthe regridder (180, 360)!

In [29]:
#historical - sst

#diff_sst_spatialmean_djf

filename='../outputs/netcdf/diff_sst/PMIP4_historical_diff_sst_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='historical'
variable_name='sst_spatialmean_djf'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)
diff_list

filename='../outputs/netcdf/diff_amm_pattern/PMIP3_historical_diff_amm_pattern_mon.nc'
pmip_v='PMIP3'
experiment_name='historical'
variable_name='amm_pattern_mon'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)
diff_list

[]

In [ ]:
#ensemble difference for gpcp (pr)
def ensemble_diffence(pmip_v,experiment_name,variable_name,filename):
    modellist=[]
    lats=np.arange(-89.5, 90., 1.0)
    lons=np.arange(-179.5, 180, 1.0)
    grid_1x1= xr.Dataset({'lat': (['lat'],lats),
                         'lon': (['lon'],lons)})
    # Use the bash script to find files with the specified variable
    A_dict=ensemble_members_dict(variable_name,experiment_name)
    B_dict=ensemble_members_dict(variable_name,'gpcp')

    #how many of the models are from this generation?
    n_gcms_of_pmip_v=0
    for gcm in A_dict:
        if gcm in B_dict:
            if gcm in pmip[pmip_v]:
                n_gcms_of_pmip_v=n_gcms_of_pmip_v+1

    #Set up the holding array
    gcms=np.arange(0,n_gcms_of_pmip_v,1)  
    data=np.full((n_gcms_of_pmip_v,len(lats),len(lons)),np.nan)
 
    #loop over models
    n=0
    for gcm in A_dict:
        if gcm in B_dict:
            if gcm in pmip[pmip_v]:
                modellist.append(gcm)
                expt_a_file=xr.open_dataset(A_dict.get(gcm),decode_times=False)
                expt_a=expt_a_file[variable_name]
                expt_b_file=xr.open_dataset(B_dict.get(gcm),decode_times=False)
                expt_b=expt_b_file[variable_name]
                diff=expt_a-expt_b
                this_regridder=xe.Regridder(expt_a_file,grid_1x1,'bilinear', reuse_weights=False,periodic=True) 
                diff_1x1=this_regridder(diff) 
                expt_a_file.close()
                expt_b_file.close()
                data[n,:,:]=diff_1x1
                n=n+1

    #write out dataset        
    ds=xr.Dataset({variable_name:(["gcms","lat","lon"],data),
                   "modellist":(["gcms"],modellist)},
                 coords={'lat': (['lat'],lats),
                         'lon': (['lon'],lons),
                         'gcms':(['gcms'],gcms)})
    ds.to_netcdf(filename)      

    return modellist

In [ ]:
#piControl - pr

#diff_pr_spatialmean_djf

filename='../outputs/netcdf/diff_pr/PMIP4_piControl_diff_pr_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='piControl'
variable_name='pr_spatialmean_djf'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)
diff_list

#diff_pr_spatialmean_jja
filename='../outputs/netcdf/diff_pr/PMIP4_piControl_diff_pr_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='piControl'
variable_name='pr_spatialmean_jja'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)
diff_list


In [ ]:
#historical - pr

#diff_pr_spatialmean_djf

filename='../outputs/netcdf/diff_pr/PMIP4_historical_diff_pr_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='historical'
variable_name='pr_spatialmean_djf'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)
diff_list

#diff_pr_spatialmean_jja
filename='../outputs/netcdf/diff_pr/PMIP4_historical_diff_pr_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='historical'
variable_name='pr_spatialmean_jja'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)
diff_list
